# Loading Llama-2 model

In [ ]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.5 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Create Transformers pipeline for inferencing
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 1min 2s, sys: 29 s, total: 1min 31s
Wall time: 2min 9s


In [ ]:
# Create Transformers pipeline for inferencing on single GPU
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                  device_map="cuda:0"
                                )

In [ ]:
# Create Transformers pipeline for inferencing on both CPU & GPU
%%time
pipeline = transformers.pipeline(
                                  "text-generation",
                                  model=model_id,
                                  device_map="auto"
                                )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 17.7 s, sys: 15.7 s, total: 33.4 s
Wall time: 15.6 s


In [ ]:
prompt = 'Tell me about AI'

In [ ]:
%%time
sequences = pipeline(
    prompt,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: Tell me about AI in healthcare and its potential impact on the industry.
AI in healthcare is a rapidly growing field that has the potential to transform the industry in several ways. Here are some of the ways AI is being used in healthcare and its potential impact:
1. Predictive analytics: AI algorithms can analyze large amounts of data to identify patterns and predict patient outcomes, such as the likelihood of disease progression or response to treatment. This can help healthcare providers make more informed decisions and improve patient care.
2. Medical imaging: AI can be used to analyze medical images such as X-rays and MRIs to identify abnormalities and diagnose conditions. This can help doctors to make more accurate diagnoses and improve treatment outcomes.
3. Personalized medicine: AI can help personalize treatment plans for individual patients based on their unique genetic profiles and medical histories. This can lead
CPU times: user 10min 56s, sys: 3.25 s, total: 10min

# loading 4-bit models using bitsandbytes

In [ ]:
#restart after installation
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00


In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
model_id = "meta-llama/Llama-2-13b-chat-hf"

In [ ]:
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config,
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 35 s, sys: 38.9 s, total: 1min 13s
Wall time: 3min 37s


In [ ]:
prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n'

In [ ]:
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [ ]:
input_ids

tensor([[    1,   306, 23289,   376, 20130,  5086,  9178, 29908,   322,   376,
         29933,   392,   310, 25522,  1642,  1938,   366,   505,   738,  6907,
           800,   310,   916,  3697,   306,  1795,   763, 29973,    13]])

In [ ]:
%%time
with torch.no_grad():
  output = model.generate(input_ids=input_ids,
                          max_new_tokens=2048,
                          temperature=0.6)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


CPU times: user 1min 2s, sys: 263 ms, total: 1min 2s
Wall time: 44.9 s


In [ ]:
%%time
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

  Sure! Based on your interest in "Breaking Bad" and "Band of Brothers", here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This gritty HBO drama delves into the drug trade in Baltimore and its impact on the city and its residents.
3. "Narcos" - This Netflix series tells the true story of Pablo Escobar, the infamous Colombian drug lord, and the DEA agents tasked with bringing him down.
4. "Peaky Blinders" - This BBC series is set in post-World War I England and follows a gangster family as they rise to power in the criminal underworld.
5. "Sons of Anarchy" - This FX series follows the lives of a motorcycle club in California as they engage in illegal activities and deal with internal conflicts.
6

# Loading GGUF Models

In [ ]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.3 MB/s eta 0:00:00


In [ ]:
from ctransformers import AutoModelForCausalLM

In [ ]:
quantized_model_id = "TheBloke/Llama-2-13B-chat-GGUF"

In [ ]:
%%time
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF",
                                           model_file="llama-2-7b-chat.Q4_K_M.gguf",
                                           gpu_layers=0)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 151 ms, sys: 156 ms, total: 308 ms
Wall time: 2.47 s


In [ ]:
%%time
print(llm("AI is going to"))

 change the world, but it's not all doom and gloom. literally.
It can be used for good or ill, depending on how it’s created, who uses it, and how it’s regulated.
Here are some ways AI can positively impact society:
1. Healthcare: AI can help diagnose diseases more accurately and quickly than humans, leading to better patient outcomes. It can also help with drug discovery and personalized treatment plans.
2. Transportation: Self-driving cars and trucks can reduce the number of accidents caused by human error, improve traffic flow, and increase mobility for the elderly and disabled.
3. Education: AI can help personalize learning experiences for students, making education more efficient and effective. It can also help teachers identify students who need extra support.
4. Energy: AI can optimize energy consumption and production, leading to more efficient use of resources and reduced carbon emissions.
5. Manufacturing: AI can improve the efficiency and speed of manufacturing processes, re

# Loading GPTQ Models

In [ ]:
!pip install auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.9/398.9 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00


In [ ]:
# install auto-gptq from source
# !pip3 uninstall -y auto-gptq
# !git clone https://github.com/PanQiWei/AutoGPTQ
# !pip3 install ./AutoGPTQ

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "TheBloke/Llama-2-13B-chat-GPTQ"

In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

CPU times: user 19.4 s, sys: 14.4 s, total: 33.8 s
Wall time: 51.1 s


In [ ]:
%%time
prompt = "Tell me about AI"

input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<s> Tell me about AI in the workplace

Artificial intelligence (AI) is transforming the modern workplace in a number of ways. Here are some key trends and insights related to AI in the workplace:

1. Automation: AI is being used to automate repetitive and mundane tasks, freeing up employees to focus on more strategic and creative work. For example, AI-powered tools can help with data entry, scheduling, and customer service.
2. Enhanced decision-making: AI can help employees make better decisions by providing data-driven insights and predictions. For example, AI can analyze large amounts of data to identify trends and patterns that may not be apparent to human analysts.
3. Personalization: AI can help personalize the work experience for employees. For example, AI-powered tools can help employees find relevant information, such as company policies or benefits, more easily.
4. Improved customer experience: AI can help improve the customer experience by providing personalized support and s